In [3]:
#imports 

from os.path import expanduser
import numpy as np
import tensorflow
import sys


In [25]:
def extract_data(filename):
    relative_path = "../../data/"
    data = open(expanduser(relative_path+filename),"rb")
    data_list = []
    for line in data:
        data_list.append(line)
    return np.asarray(data_list)

def preprocess_data(data):
    for row in data:
        for i in range(len(row)):
            row[i]

def decoder(hidden_size, input_data):
    rnn_cell = tf.nn.rnn_cell.BasicRNNCell(hidden_size)
    initial_state = rnn_cell.zero_state(batch_size, dtype=tf.float32)
    outputs, state = tf.nn.dynamic_rnn(rnn_cell, input_data, initial_state = initial_state, dtype=tf.float32)
    return outputs

def encoder(hidden_size, input_data):
    rnn_cell = tf.nn.rnn_cell.BasicRNNCell(hidden_size)
    initial_state = rnn_cell.zero_state(batch_size, dtype=tf.float32)
    outputs, state = tf.nn.dynamic_rnn(rnn_cell, input_data, initial_state = initial_state, dtype=tf.float32)
    return outputs



IndentationError: expected an indented block (<ipython-input-25-27b99c815c01>, line 12)

In [18]:
#extract data
train_english = extract_data("train.10k.en")
train_german = extract_data("train.10k.de")
valid_english = extract_data("valid.100.en")
valid_german = extract_data("valid.100.de")


In [29]:
valid_german[0]

'Eine republikanische Strategie , um der Wiederwahl von Obama entgegenzutreten\n'